In [1]:
import math,time
import torch
from fcos import FCOSDetector
from VOC_dataset import VOCDataset
from torch.utils.tensorboard import SummaryWriter

In [2]:
train_dataset=VOCDataset("f:/Dataset/VOC/VOC2007/",resize_size=[512,800],split='train')
val_dataset=VOCDataset("f:/Dataset/VOC/VOC2007/",resize_size=[512,800],split='val')

INFO=====>voc dataset init finished  ! !
INFO=====>voc dataset init finished  ! !


In [3]:
model=FCOSDetector(mode="training").cuda()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)

In [4]:
BATCH_SIZE=6
EPOCHS=30
WARMPUP_STEPS_RATIO=0.12

train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,collate_fn=train_dataset.collate_fn)
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=True,collate_fn=val_dataset.collate_fn)

steps_per_epoch=len(train_dataset)//BATCH_SIZE
TOTAL_STEPS=steps_per_epoch*EPOCHS
WARMPUP_STEPS=TOTAL_STEPS*WARMPUP_STEPS_RATIO


GLOBAL_STEPS=1
LR_INIT=5e-5
LR_END=1e-6


writer=SummaryWriter(log_dir="./logs")

def lr_func():
    if GLOBAL_STEPS<WARMPUP_STEPS:
        lr=GLOBAL_STEPS/WARMPUP_STEPS*LR_INIT
    else:
        lr=LR_END+0.5*(LR_INIT-LR_END)*(
            (1+math.cos((GLOBAL_STEPS-WARMPUP_STEPS)/(TOTAL_STEPS-WARMPUP_STEPS)*math.pi))
        )
    return float(lr)

In [5]:
model.train()

INFO===>success frozen BN
INFO===>success frozen backbone stage1


FCOSDetector(
  (fcos_body): FCOS(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inp

In [6]:
for epoch in range(EPOCHS):
    for epoch_step,data in enumerate(train_loader):

        batch_imgs,batch_boxes,batch_classes=data
        batch_imgs=batch_imgs.cuda()
        batch_boxes=batch_boxes.cuda()
        batch_classes=batch_classes.cuda()

        lr=lr_func()
        for param in optimizer.param_groups:
            param['lr']=lr
        
        start_time=time.time()

        optimizer.zero_grad()
        losses=model([batch_imgs,batch_boxes,batch_classes])
        loss=losses[-1]
        loss.backward()
        optimizer.step()

        end_time=time.time()
        cost_time=int((end_time-start_time)*1000)

        print("global_steps:%d epoch:%d steps:%d/%d cls_loss:%.4f cnt_loss:%.4f reg_loss:%.4f cost_time:%dms lr=%.4e"%\
            (GLOBAL_STEPS,epoch+1,epoch_step+1,steps_per_epoch,losses[0],losses[1],losses[2],cost_time,lr))
        
        writer.add_scalar("loss/cls_loss",losses[0],global_step=GLOBAL_STEPS)
        writer.add_scalar("loss/cnt_loss",losses[1],global_step=GLOBAL_STEPS)
        writer.add_scalar("loss/reg_loss",losses[2],global_step=GLOBAL_STEPS)
        writer.add_scalar("lr",lr,global_step=GLOBAL_STEPS)

        GLOBAL_STEPS+=1
    
    torch.save(model.state_dict(),"./voc2012_512x800_epoch%d_loss%.4f.pth"%(epoch+1,loss.item()))
    
writer.close()

global_steps:1 epoch:1 steps:1/416 cls_loss:3.3097 cnt_loss:0.7509 reg_loss:0.9997 cost_time:1349ms lr=3.3387e-08
global_steps:2 epoch:1 steps:2/416 cls_loss:3.4987 cnt_loss:0.7172 reg_loss:0.9999 cost_time:268ms lr=6.6774e-08
global_steps:3 epoch:1 steps:3/416 cls_loss:3.4394 cnt_loss:0.7722 reg_loss:0.9999 cost_time:333ms lr=1.0016e-07
global_steps:4 epoch:1 steps:4/416 cls_loss:3.3357 cnt_loss:0.7207 reg_loss:1.0001 cost_time:291ms lr=1.3355e-07
global_steps:5 epoch:1 steps:5/416 cls_loss:3.1899 cnt_loss:0.7458 reg_loss:0.9999 cost_time:789ms lr=1.6693e-07
global_steps:6 epoch:1 steps:6/416 cls_loss:3.5205 cnt_loss:0.7136 reg_loss:0.9996 cost_time:263ms lr=2.0032e-07
global_steps:7 epoch:1 steps:7/416 cls_loss:3.3563 cnt_loss:0.7239 reg_loss:0.9998 cost_time:261ms lr=2.3371e-07
global_steps:8 epoch:1 steps:8/416 cls_loss:3.2350 cnt_loss:0.7230 reg_loss:0.9999 cost_time:308ms lr=2.6709e-07
global_steps:9 epoch:1 steps:9/416 cls_loss:3.2987 cnt_loss:0.7406 reg_loss:0.9998 cost_time:34

global_steps:73 epoch:1 steps:73/416 cls_loss:2.8868 cnt_loss:0.6561 reg_loss:0.9998 cost_time:320ms lr=2.4372e-06
global_steps:74 epoch:1 steps:74/416 cls_loss:3.0496 cnt_loss:0.6672 reg_loss:0.9985 cost_time:271ms lr=2.4706e-06
global_steps:75 epoch:1 steps:75/416 cls_loss:2.9862 cnt_loss:0.6752 reg_loss:0.9997 cost_time:314ms lr=2.5040e-06
global_steps:76 epoch:1 steps:76/416 cls_loss:3.0018 cnt_loss:0.6654 reg_loss:0.9979 cost_time:279ms lr=2.5374e-06
global_steps:77 epoch:1 steps:77/416 cls_loss:2.7760 cnt_loss:0.6399 reg_loss:0.9998 cost_time:313ms lr=2.5708e-06
global_steps:78 epoch:1 steps:78/416 cls_loss:2.5972 cnt_loss:0.6568 reg_loss:0.9987 cost_time:268ms lr=2.6042e-06
global_steps:79 epoch:1 steps:79/416 cls_loss:3.2043 cnt_loss:0.6383 reg_loss:0.9991 cost_time:270ms lr=2.6376e-06
global_steps:80 epoch:1 steps:80/416 cls_loss:2.6305 cnt_loss:0.6674 reg_loss:0.9996 cost_time:321ms lr=2.6709e-06
global_steps:81 epoch:1 steps:81/416 cls_loss:2.8294 cnt_loss:0.6628 reg_loss:0.

global_steps:144 epoch:1 steps:144/416 cls_loss:2.4499 cnt_loss:0.6648 reg_loss:0.9514 cost_time:366ms lr=4.8077e-06
global_steps:145 epoch:1 steps:145/416 cls_loss:2.1866 cnt_loss:0.6526 reg_loss:0.9954 cost_time:327ms lr=4.8411e-06
global_steps:146 epoch:1 steps:146/416 cls_loss:1.8545 cnt_loss:0.6533 reg_loss:0.9926 cost_time:283ms lr=4.8745e-06
global_steps:147 epoch:1 steps:147/416 cls_loss:2.0760 cnt_loss:0.6487 reg_loss:0.9920 cost_time:308ms lr=4.9079e-06
global_steps:148 epoch:1 steps:148/416 cls_loss:2.2424 cnt_loss:0.6543 reg_loss:0.9836 cost_time:381ms lr=4.9412e-06
global_steps:149 epoch:1 steps:149/416 cls_loss:1.6404 cnt_loss:0.6510 reg_loss:0.9733 cost_time:288ms lr=4.9746e-06
global_steps:150 epoch:1 steps:150/416 cls_loss:2.2078 cnt_loss:0.6626 reg_loss:0.9810 cost_time:342ms lr=5.0080e-06
global_steps:151 epoch:1 steps:151/416 cls_loss:2.6445 cnt_loss:0.6671 reg_loss:1.0003 cost_time:347ms lr=5.0414e-06
global_steps:152 epoch:1 steps:152/416 cls_loss:1.3728 cnt_loss:

global_steps:215 epoch:1 steps:215/416 cls_loss:1.3946 cnt_loss:0.6527 reg_loss:0.8647 cost_time:313ms lr=7.1782e-06
global_steps:216 epoch:1 steps:216/416 cls_loss:1.4960 cnt_loss:0.6360 reg_loss:0.8976 cost_time:313ms lr=7.2115e-06
global_steps:217 epoch:1 steps:217/416 cls_loss:1.3217 cnt_loss:0.6539 reg_loss:0.8611 cost_time:358ms lr=7.2449e-06
global_steps:218 epoch:1 steps:218/416 cls_loss:1.1632 cnt_loss:0.6571 reg_loss:0.8761 cost_time:332ms lr=7.2783e-06
global_steps:219 epoch:1 steps:219/416 cls_loss:1.6708 cnt_loss:0.6622 reg_loss:0.9123 cost_time:393ms lr=7.3117e-06
global_steps:220 epoch:1 steps:220/416 cls_loss:1.9632 cnt_loss:0.6630 reg_loss:0.9103 cost_time:388ms lr=7.3451e-06
global_steps:221 epoch:1 steps:221/416 cls_loss:1.3925 cnt_loss:0.6465 reg_loss:0.8471 cost_time:325ms lr=7.3785e-06
global_steps:222 epoch:1 steps:222/416 cls_loss:1.7520 cnt_loss:0.6468 reg_loss:0.7142 cost_time:291ms lr=7.4119e-06
global_steps:223 epoch:1 steps:223/416 cls_loss:1.1794 cnt_loss:

global_steps:286 epoch:1 steps:286/416 cls_loss:0.8726 cnt_loss:0.6606 reg_loss:0.6705 cost_time:358ms lr=9.5486e-06
global_steps:287 epoch:1 steps:287/416 cls_loss:1.0161 cnt_loss:0.6616 reg_loss:0.6814 cost_time:324ms lr=9.5820e-06
global_steps:288 epoch:1 steps:288/416 cls_loss:1.4715 cnt_loss:0.6478 reg_loss:0.8444 cost_time:258ms lr=9.6154e-06
global_steps:289 epoch:1 steps:289/416 cls_loss:1.2626 cnt_loss:0.6384 reg_loss:0.7393 cost_time:310ms lr=9.6488e-06
global_steps:290 epoch:1 steps:290/416 cls_loss:1.3295 cnt_loss:0.6553 reg_loss:0.7065 cost_time:358ms lr=9.6822e-06
global_steps:291 epoch:1 steps:291/416 cls_loss:1.1048 cnt_loss:0.6387 reg_loss:0.8112 cost_time:266ms lr=9.7155e-06
global_steps:292 epoch:1 steps:292/416 cls_loss:1.3879 cnt_loss:0.6218 reg_loss:0.7170 cost_time:370ms lr=9.7489e-06
global_steps:293 epoch:1 steps:293/416 cls_loss:1.3307 cnt_loss:0.6544 reg_loss:0.7086 cost_time:331ms lr=9.7823e-06
global_steps:294 epoch:1 steps:294/416 cls_loss:1.5059 cnt_loss:

...

global_steps:12176 epoch:30 steps:83/416 cls_loss:0.0022 cnt_loss:0.5973 reg_loss:0.0881 cost_time:363ms lr=1.0926e-06
global_steps:12177 epoch:30 steps:84/416 cls_loss:0.0025 cnt_loss:0.5990 reg_loss:0.0295 cost_time:238ms lr=1.0920e-06
global_steps:12178 epoch:30 steps:85/416 cls_loss:0.0042 cnt_loss:0.6019 reg_loss:0.0361 cost_time:263ms lr=1.0914e-06
global_steps:12179 epoch:30 steps:86/416 cls_loss:0.0046 cnt_loss:0.5708 reg_loss:0.0745 cost_time:317ms lr=1.0908e-06
global_steps:12180 epoch:30 steps:87/416 cls_loss:0.0031 cnt_loss:0.6063 reg_loss:0.0490 cost_time:359ms lr=1.0902e-06
global_steps:12181 epoch:30 steps:88/416 cls_loss:0.0041 cnt_loss:0.5879 reg_loss:0.0598 cost_time:314ms lr=1.0896e-06
global_steps:12182 epoch:30 steps:89/416 cls_loss:0.0032 cnt_loss:0.5816 reg_loss:0.0333 cost_time:234ms lr=1.0890e-06
global_steps:12183 epoch:30 steps:90/416 cls_loss:0.0011 cnt_loss:0.5966 reg_loss:0.0405 cost_time:263ms lr=1.0884e-06
global_steps:12184 epoch:30 steps:91/416 cls_los

global_steps:12245 epoch:30 steps:152/416 cls_loss:0.0043 cnt_loss:0.5976 reg_loss:0.0632 cost_time:270ms lr=1.0553e-06
global_steps:12246 epoch:30 steps:153/416 cls_loss:0.0058 cnt_loss:0.5790 reg_loss:0.0557 cost_time:236ms lr=1.0549e-06
global_steps:12247 epoch:30 steps:154/416 cls_loss:0.0042 cnt_loss:0.6030 reg_loss:0.0546 cost_time:266ms lr=1.0544e-06
global_steps:12248 epoch:30 steps:155/416 cls_loss:0.0014 cnt_loss:0.5940 reg_loss:0.0282 cost_time:355ms lr=1.0539e-06
global_steps:12249 epoch:30 steps:156/416 cls_loss:0.0077 cnt_loss:0.5986 reg_loss:0.0430 cost_time:334ms lr=1.0535e-06
global_steps:12250 epoch:30 steps:157/416 cls_loss:0.0054 cnt_loss:0.6012 reg_loss:0.0681 cost_time:268ms lr=1.0530e-06
global_steps:12251 epoch:30 steps:158/416 cls_loss:0.0030 cnt_loss:0.5870 reg_loss:0.0596 cost_time:333ms lr=1.0525e-06
global_steps:12252 epoch:30 steps:159/416 cls_loss:0.0035 cnt_loss:0.5900 reg_loss:0.0581 cost_time:292ms lr=1.0521e-06
global_steps:12253 epoch:30 steps:160/41

global_steps:12314 epoch:30 steps:221/416 cls_loss:0.0050 cnt_loss:0.5905 reg_loss:0.0585 cost_time:333ms lr=1.0276e-06
global_steps:12315 epoch:30 steps:222/416 cls_loss:0.0026 cnt_loss:0.5759 reg_loss:0.0258 cost_time:355ms lr=1.0273e-06
global_steps:12316 epoch:30 steps:223/416 cls_loss:0.0031 cnt_loss:0.5864 reg_loss:0.0323 cost_time:238ms lr=1.0270e-06
global_steps:12317 epoch:30 steps:224/416 cls_loss:0.0066 cnt_loss:0.5940 reg_loss:0.0963 cost_time:268ms lr=1.0266e-06
global_steps:12318 epoch:30 steps:225/416 cls_loss:0.0034 cnt_loss:0.5915 reg_loss:0.0443 cost_time:313ms lr=1.0263e-06
global_steps:12319 epoch:30 steps:226/416 cls_loss:0.0019 cnt_loss:0.5801 reg_loss:0.0338 cost_time:313ms lr=1.0260e-06
global_steps:12320 epoch:30 steps:227/416 cls_loss:0.0045 cnt_loss:0.5978 reg_loss:0.0286 cost_time:265ms lr=1.0257e-06
global_steps:12321 epoch:30 steps:228/416 cls_loss:0.0018 cnt_loss:0.5732 reg_loss:0.0218 cost_time:249ms lr=1.0253e-06
global_steps:12322 epoch:30 steps:229/41

global_steps:12383 epoch:30 steps:290/416 cls_loss:0.0046 cnt_loss:0.5862 reg_loss:0.0416 cost_time:359ms lr=1.0094e-06
global_steps:12384 epoch:30 steps:291/416 cls_loss:0.0046 cnt_loss:0.5960 reg_loss:0.0400 cost_time:358ms lr=1.0092e-06
global_steps:12385 epoch:30 steps:292/416 cls_loss:0.0063 cnt_loss:0.6057 reg_loss:0.0725 cost_time:316ms lr=1.0090e-06
global_steps:12386 epoch:30 steps:293/416 cls_loss:0.0032 cnt_loss:0.6062 reg_loss:0.0685 cost_time:335ms lr=1.0089e-06
global_steps:12387 epoch:30 steps:294/416 cls_loss:0.0014 cnt_loss:0.5794 reg_loss:0.0203 cost_time:287ms lr=1.0087e-06
global_steps:12388 epoch:30 steps:295/416 cls_loss:0.0052 cnt_loss:0.5916 reg_loss:0.0767 cost_time:240ms lr=1.0085e-06
global_steps:12389 epoch:30 steps:296/416 cls_loss:0.0032 cnt_loss:0.5911 reg_loss:0.0512 cost_time:354ms lr=1.0083e-06
global_steps:12390 epoch:30 steps:297/416 cls_loss:0.0059 cnt_loss:0.5906 reg_loss:0.0820 cost_time:266ms lr=1.0081e-06
global_steps:12391 epoch:30 steps:298/41

global_steps:12452 epoch:30 steps:359/416 cls_loss:0.0009 cnt_loss:0.5949 reg_loss:0.0250 cost_time:313ms lr=1.0008e-06
global_steps:12453 epoch:30 steps:360/416 cls_loss:0.0033 cnt_loss:0.5763 reg_loss:0.0513 cost_time:268ms lr=1.0007e-06
global_steps:12454 epoch:30 steps:361/416 cls_loss:0.0016 cnt_loss:0.5981 reg_loss:0.0330 cost_time:331ms lr=1.0007e-06
global_steps:12455 epoch:30 steps:362/416 cls_loss:0.0030 cnt_loss:0.5827 reg_loss:0.0345 cost_time:266ms lr=1.0006e-06
global_steps:12456 epoch:30 steps:363/416 cls_loss:0.0042 cnt_loss:0.5924 reg_loss:0.0578 cost_time:356ms lr=1.0006e-06
global_steps:12457 epoch:30 steps:364/416 cls_loss:0.0022 cnt_loss:0.5934 reg_loss:0.0360 cost_time:308ms lr=1.0005e-06
global_steps:12458 epoch:30 steps:365/416 cls_loss:0.0027 cnt_loss:0.5796 reg_loss:0.0384 cost_time:357ms lr=1.0005e-06
global_steps:12459 epoch:30 steps:366/416 cls_loss:0.0109 cnt_loss:0.5903 reg_loss:0.0983 cost_time:264ms lr=1.0004e-06
global_steps:12460 epoch:30 steps:367/41